# Random Forest(1)

의사결정트리를 사용하는 가장 대표적인 배깅 모델

의사결정트리의 단점(과적합이 자주 발생)을 보완하고 장점은 유지한다.

최근 XGBoost, LightGBM, CatBoost와 함께 주목받는 알고리즘 중 하나.


## 01. 패키지 참조

In [2]:
from matplotlib import pyplot as plt
import seaborn as sb
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


# 데이터 불균형 해소를 위한 패키지
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

## 02. 데이터 가져오기

|필드이름|설명|
|---|---|
|target| 타겟(종속)변수 `Class_1~Class_9`|
|feat_1~feat_93|설명(독립)변수|

In [19]:
origin = pd.read_csv("./otto_train.csv")

In [20]:
origin = origin.drop(['id'],axis=1)

In [21]:
origin.head()

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,1,0,0,1,6,1,5,0,0,1,...,0,1,2,0,0,0,0,0,0,Class_1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [22]:
origin['target'] = origin['target'].map({
     "Class_1":0,
     "Class_2":1,
     "Class_3":2,
     "Class_4":3,
     "Class_5":4,
     "Class_6":5,
     "Class_7":6,
     "Class_8":7,
     "Class_9":8
})
origin['target'].value_counts().sort_index()

target
0     1929
1    16122
2     8004
3     2691
4     2739
5    14135
6     2839
7     8464
8     4955
Name: count, dtype: int64

In [23]:
x = origin.drop("target", axis=1)
y = origin['target']
x.shape, y.shape

((61878, 93), (61878,))

In [24]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size = 0.3 , random_state = 111)
x_train.shape, x_test.shape, y_train.shape, y_test.shape


((43314, 93), (18564, 93), (43314,), (18564,))

### 데이터 불균형 해소

1) Under Sampling 방식 - Random Under Sampler

많은 비율을 차지하는 다수 집단에서 일부만 샘플링하는 방식

소수 집단의 데이터가 어느 정도 확보되었다고 여겨질 때, 다수 집단의 데이터를 줄여서 균형을 맞춘다.

다수 집단의 유의미한 데이터를 손실할 수 있다는 단점이 있다.

`sampling_strategy` 파라미터

| 값 | 설명 |
| --- | --- |
| `majority` | 다수 클래스만 다시 샘플링 |
| `not majority` | `다수 아님`  - 다수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `not minority` | `소수 아님`  - 소수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `all` | 모든 클래스를 다시 샘플링 |
| `auto` | 자동 처리 |


In [26]:
undersampler = RandomUnderSampler(sampling_strategy="majority", random_state = 111)
x_under, y_under = undersampler.fit_resample(x_train, y_train)
print(x_under.shape, y_under.shape)

y_under.value_counts().sort_index()

(33356, 93) (33356,)


target
0    1339
1    1339
2    5615
3    1870
4    1905
5    9978
6    1967
7    5863
8    3480
Name: count, dtype: int64

### Over Sampling - Random Over Sampler

소수 집단에서 복원 추출을 수행하는 방법

언더 샘플링처럼 데이터 중 일부를 취하는 것은 아니기 때문에 데이터 손실은 발생하지 않지만, 동일한 데이터를 여러번 학습 데이터에 포함시키므로 학습 정확도는 높지만 과적합 리스크가 크다.

`sampling_strategy` 파라미터

| 값 | 설명 |
| --- | --- |
| `minority` | 소수 클래스만 다시 샘플링 |
| `not majority` | `다수 아님` - 다수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `not minority`| `소수 아님`- 소수 클래스를 제외한 모든 클래스르 다시 샘플링 |
| `all` | 모든 클래스를 다시 샘플링 |
| `auto` | 자동 처리|

In [27]:
oversampler = RandomOverSampler(sampling_strategy="minority", random_state = 111)
x_over, y_over = oversampler.fit_resample(x_train, y_train)
print(x_over.shape, y_over.shape)

y_over.value_counts().sort_index()

(53272, 93) (53272,)


target
0    11297
1    11297
2     5615
3     1870
4     1905
5     9978
6     1967
7     5863
8     3480
Name: count, dtype: int64

### Over Sampling - Random Over Sampler

소수 집단의 데이터를 바탕으로 새로운 데이터를 생성.

단순히 소수 집단의 데이터를 복원 추출하는 것이 아니라 소수 집단 데이터를 분석해 어떤 특징이 있는지 살피고 그와 유사한 패턴을 갖는 가짜 데이터를 생성한다.


`sampling_strategy` 파라미터

| 값 | 설명 |
| --- | --- |
| `minority` | 소수 클래스만 다시 샘플링 |
| `not majority` | `다수 아님` - 다수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `not minority`| `소수 아님`- 소수 클래스를 제외한 모든 클래스르 다시 샘플링 |
| `all` | 모든 클래스를 다시 샘플링 |
| `auto` | 자동 처리|

혹은 실수 타입으로 설정할 경우 샘플 수의 비율을 의미

`k_neighbors` 파라미터 (int)

합성 샘플을 생성하는데 사용할 샘플의 가장 가까운 이웃 수 (기본값=5)

In [28]:
# "sampling_strategy" can be a float only when the type of target is binary
smote_sampler = SMOTE(sampling_strategy="minority", random_state=111)
x_sm, y_sm = smote_sampler.fit_resample(x_train, y_train)
print(x_sm.shape, y_sm.shape)

y_sm.value_counts().sort_index()

(53272, 93) (53272,)


target
0    11297
1    11297
2     5615
3     1870
4     1905
5     9978
6     1967
7     5863
8     3480
Name: count, dtype: int64

## 03. 랜덤포레스트 모델 적합
#### 단일 모델 만들기

In [31]:
rfc = RandomForestClassifier(n_estimators=20, max_depth = 5, random_state=111)

# 원본 데이터로 학습 진행
rfc.fit(x_train, y_train)
print("훈련 정확도: ", rfc.score(x_train, y_train))

# 언더샘플링 데이터로 학습 진행
rfc.fit(x_under, y_under)
print("훈련 정확도: ", rfc.score(x_under, y_under))

# 오버샘플링 데이터로 학습 진행행
rfc.fit(x_over, y_over)
print("훈련 정확도: ", rfc.score(x_over, y_over))

# SMOTE 데이터로 학습 진행행
rfc.fit(x_sm, y_sm)
print("훈련 정확도: ", rfc.score(x_sm, y_sm))

print("테스트 정확도:", rfc.score(x_test, y_test))

훈련 정확도:  0.5992750611811424
훈련 정확도:  0.6641383858975897
훈련 정확도:  0.609644841567803
훈련 정확도:  0.6134367022075387
테스트 정확도: 0.5347985347985348


In [32]:
rfc = RandomForestClassifier(n_estimators=50, max_depth = 5, random_state=111)

# 원본 데이터로 학습 진행
rfc.fit(x_train, y_train)
print("훈련 정확도: ", rfc.score(x_train, y_train))

# 언더샘플링 데이터로 학습 진행
rfc.fit(x_under, y_under)
print("훈련 정확도: ", rfc.score(x_under, y_under))

# 오버샘플링 데이터로 학습 진행행
rfc.fit(x_over, y_over)
print("훈련 정확도: ", rfc.score(x_over, y_over))

# SMOTE 데이터로 학습 진행행
rfc.fit(x_sm, y_sm)
print("훈련 정확도: ", rfc.score(x_sm, y_sm))

print("테스트 정확도:", rfc.score(x_test, y_test))

훈련 정확도:  0.6081405550168537
훈련 정확도:  0.6797577647199904
훈련 정확도:  0.6197627271362066
훈련 정확도:  0.6118035741102268
테스트 정확도: 0.5322128851540616


In [33]:
rfc = RandomForestClassifier(n_estimators=100, max_depth = 30, random_state=111)

# 원본 데이터로 학습 진행
rfc.fit(x_train, y_train)
print("훈련 정확도: ", rfc.score(x_train, y_train))

# 언더샘플링 데이터로 학습 진행
rfc.fit(x_under, y_under)
print("훈련 정확도: ", rfc.score(x_under, y_under))

# 오버샘플링 데이터로 학습 진행행
rfc.fit(x_over, y_over)
print("훈련 정확도: ", rfc.score(x_over, y_over))

# SMOTE 데이터로 학습 진행행
rfc.fit(x_sm, y_sm)
print("훈련 정확도: ", rfc.score(x_sm, y_sm))

print("테스트 정확도:", rfc.score(x_test, y_test))

훈련 정확도:  0.9742577457634944
훈련 정확도:  0.9816224967022424
훈련 정확도:  0.97809355759123
훈련 정확도:  0.9780372428292536
테스트 정확도: 0.7854988149105796


In [34]:
rfc = RandomForestClassifier(n_estimators=100, max_depth = 100, random_state=111)

# 원본 데이터로 학습 진행
rfc.fit(x_train, y_train)
print("훈련 정확도: ", rfc.score(x_train, y_train))

# 언더샘플링 데이터로 학습 진행
rfc.fit(x_under, y_under)
print("훈련 정확도: ", rfc.score(x_under, y_under))

# 오버샘플링 데이터로 학습 진행행
rfc.fit(x_over, y_over)
print("훈련 정확도: ", rfc.score(x_over, y_over))

# SMOTE 데이터로 학습 진행행
rfc.fit(x_sm, y_sm)
print("훈련 정확도: ", rfc.score(x_sm, y_sm))

print("테스트 정확도:", rfc.score(x_test, y_test))

훈련 정확도:  1.0
훈련 정확도:  1.0
훈련 정확도:  1.0
훈련 정확도:  0.9999624568253491
테스트 정확도: 0.7932557638439991


#### 하이퍼 파라미터 튜닝

In [36]:
rfc = RandomForestClassifier(random_state =111)

params = {
    "n_estimators": [20, 50 ,100],
    "max_depth" : [5, 30, 100]
}

grid = GridSearchCV(rfc, param_grid=params, cv=5, n_jobs= -1)
grid.fit(x_train, y_train)

print("최적의 하이퍼 파라미터:", grid.best_params_)
print("최대 훈련 정확도:", grid.best_score_)

y_pred = grid.best_estimator_.predict(x_test)
print("최대 검증 정확도:", accuracy_score(y_test, y_pred))

result_df = pd.DataFrame(grid.cv_results_['params'])
result_df['mean_test_score'] = grid.cv_results_['mean_test_score']
result_df.sort_values(by="mean_test_score", ascending=False)

최적의 하이퍼 파라미터: {'max_depth': 100, 'n_estimators': 100}
최대 훈련 정확도: 0.8007572951556622
최대 검증 정확도: 0.8035444947209653


,max_depth,n_estimators,mean_test_score
8,100,100,0.800757
5,30,100,0.796579
7,100,50,0.796209
4,30,50,0.794524
6,100,20,0.787136
3,30,20,0.782680
2,5,100,0.616475
1,5,50,0.609872
0,5,20,0.601792
